FAQ Chatbot For Any Topic

This project is on creating a QA Chatbot for any topic given the static webpage's url or a PDF. Krish Naik's video was used for learning how to use Ollama. Keerti P.'s tutorial was used for learning the actual code for enterprise which was slightly altered for running using Ollama (locally run llms). A web-based app was developed using REST, FAST API, and Streamlit. FAST API for app backend (built using REST). Streamlit for frontend client interface. Routes can be used between app api and llms to interact with multiple llms. However, only one llm, llama 3.1, was used. The app was hosted locally. Later, the app was not able to run smoothly. So, the app format was not used and the notebook was left as is. Another attempt to make an LLM app will be made with a different code approach.

For FAQ On Uploaded URLs, PDFs, use LangChain for Naive RAG

In [1]:
!pip install langchain_community langchainhub chromadb langchain langchain-openai

In [2]:
!pip install pypdf

In [3]:
from langchain_community.document_loaders import WebBaseLoader
import os
from langchain_community.document_loaders import PyPDFLoader
#from langchain_community.document_loaders import PlaywrightURLLoader
import os
os.environ["USER_AGENT"] = "Mozilla/5.0"

def load_document(input_path: str):
    if input_path.lower().startswith("http"):
        # Web scraping
        loader = WebBaseLoader(web_paths=[input_path])#This is only for static websites
        #loader = PlaywrightURLLoader(urls=input_path, remove_selectors=["header", "footer", "nav"])
        docs = loader.load()
    elif input_path.lower().endswith(".pdf"):
        loader = PyPDFLoader(input_path)
        docs = loader.load()
    else:
        raise ValueError("Unsupported input. Provide a valid URL or local PDF file.")

    print(docs)
    return docs


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [23]:
message = input("Enter whether you want to submit a url or a pdf: ")

if message == "url":
    user_input = input("Enter a URL: ")
else:
    user_input = input("Enter a local PDF file path: ")
try:
    documents = load_document(user_input)
except Exception as e:
    print("Error:", e)

Error: File path Concepts/Attention is All You Need.pdf is not a valid file or url


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(documents)

In [6]:
print(splits[0])
print(splits[1])

page_content='Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Hands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AI7 Weeks CourseAccess all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics Architectures: Feedforward, RNN, and CNN Mini Project - Build a Simple Neural Network Using TensorFlow Mini Project - Train an Autoencoder on the MNIST Dataset2Week 2Deep Generative Models Discriminative and Generative models Generative Adversarial Networks (GANs) Variational Autoencoders (VAEs) Probabilistic Data Generation Using VAEs Four Mini Projects using TensorFlow Metrics Visualization using TensorBoard Mini Project - Implement a GAN to Generate Handwritten Di

In [7]:
print(len(splits))

11


In [8]:
!pip install langchain-huggingface

In [9]:
!pip install sentence-transformers

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)

In [11]:
!pip install langchain-ollama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model = "llama3.1")

In [13]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

/opt/anaconda3/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [16]:
print(vectorstore._collection.count())

11


In [17]:
print(vectorstore._collection.get())

{'ids': ['23697687-a448-428e-8802-ef1566035a35', 'b6b60ef5-e387-4e84-bc96-349d5c059302', '9dfc5fbb-d654-417a-af9e-22ae398bfa28', 'ea7f3432-315f-4f8d-817c-6d1601f94125', '694e6619-48cb-4a03-815b-6fac475be1d6', '5cc4f2a1-20e7-4801-a907-0ded6eec6a64', '259aed02-9a57-4548-b6ef-48392e80e9ac', '4283bf1e-fad6-418e-931d-fd9aa5e438d2', '2453dfd1-96c6-4fa4-b426-8b322872d7f6', 'f067c529-8d4f-4223-89d6-6f04af4c0895', '5e700042-9746-4a77-9a99-1880842905d4'], 'embeddings': None, 'documents': ['Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Hands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AI7 Weeks CourseAccess all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics 

In [18]:
print("\nCollection 1 = ", vectorstore._collection.get(ids = ['aef0e8d9-d361-4fdd-9f31-1f6fe62be589'], include = ["embeddings", "documents"]))


Collection 1 =  {'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['embeddings', 'documents'], 'data': None, 'metadatas': None}


In [19]:
retriever = vectorstore.as_retriever

In [20]:
rag_chain = ({"context": retriever() | format_docs, "question": RunnablePassthrough()}
             | prompt
             | llm
             | StrOutputParser())

In [24]:
rag_chain.invoke("Are the recordings of the course available? For how long?")

'According to the provided context, it is mentioned that "Can I watch the class recordings or go through notes offline?" which implies that class recordings are available for viewing. However, there\'s no specific information on how long the recordings will be available.'